In [1]:
import pandas as pd

In [2]:
pref = pd.read_csv("COVID-19-jp/nhk_news_covid19_prefectures_daily_data.csv")
pref["日付"] = pd.to_datetime(pref["日付"] )
print(len(pref))
pref.tail()

46342


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,各地の直近1週間の人口10万人あたりの感染者数
46337,2022-09-23,47,沖縄県,639,496997,6,706,265.56
46338,2022-09-24,47,沖縄県,338,497335,5,711,243.48
46339,2022-09-25,47,沖縄県,533,497868,0,711,243.41
46340,2022-09-26,47,沖縄県,284,498152,3,714,242.73
46341,2022-09-27,47,沖縄県,568,498720,0,714,259.90


In [3]:
latest_date = pref["日付"].max().strftime("%Y-%m-%d")

In [4]:
all_jp = pref.groupby(["都道府県名"], as_index=False).sum()
del all_jp["各地の感染者数_累計"]
del all_jp["各地の死者数_累計"]
#all_jp = all_jp.rename(columns={"各地の感染者数合計":"各地の感染者数_1日ごとの発表数"})
all_jp = all_jp.rename(columns={"各地の感染者数_1日ごとの発表数":"各地の感染者数合計"})
all_jp = all_jp.rename(columns={"各地の死者数_1日ごとの発表数":"各地の死者数合計"})
all_jp.tail()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数
42,静岡県,21692,490165,739,93707.47
43,香川県,36482,141496,289,103408.08
44,高知県,38454,99691,294,100035.10
45,鳥取県,30566,62383,73,78029.18
46,鹿児島県,45356,297668,535,130464.53


## 都道府県ごとの最新データを付与

In [5]:
# 直近１週間の感染者数
pref["直近1週間の感染者数"] = pref["各地の感染者数_1日ごとの発表数"].rolling(7).sum()
pref.tail(7)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,各地の直近1週間の人口10万人あたりの感染者数,直近1週間の感染者数
46335,2022-09-21,47,沖縄県,782,495688,0,700,274.42,4027.0
46336,2022-09-22,47,沖縄県,670,496358,0,700,268.76,3944.0
46337,2022-09-23,47,沖縄県,639,496997,6,706,265.56,3897.0
46338,2022-09-24,47,沖縄県,338,497335,5,711,243.48,3573.0
46339,2022-09-25,47,沖縄県,533,497868,0,711,243.41,3572.0
46340,2022-09-26,47,沖縄県,284,498152,3,714,242.73,3562.0
46341,2022-09-27,47,沖縄県,568,498720,0,714,259.90,3814.0


### 感染者数前日比

In [6]:
import numpy as np
pref["新規感染者数前日比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change()
# 一番古い日付だけNANを設定
pref.loc[pref["日付"]==pref["日付"].min(),"新規感染者数前日比"] = np.nan
pref["直近1週間の新規感染者数前日比平均"] = pref["新規感染者数前日比"].rolling(7).mean()
pref["新規感染者数先週比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change(periods=7)

pref.tail(10)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,各地の直近1週間の人口10万人あたりの感染者数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
46332,2022-09-18,47,沖縄県,534,494296,0,698,323.21,4743.0,-0.193353,0.007261,-0.259362
46333,2022-09-19,47,沖縄県,294,494590,0,698,312.37,4584.0,-0.449438,-0.003844,-0.350993
46334,2022-09-20,47,沖縄県,316,494906,2,700,277.28,4069.0,0.074830,-0.112359,-0.619735
46335,2022-09-21,47,沖縄県,782,495688,0,700,274.42,4027.0,1.474684,0.099513,-0.050971
46336,2022-09-22,47,沖縄県,670,496358,0,700,268.76,3944.0,-0.143223,0.091362,-0.110226
46337,2022-09-23,47,沖縄県,639,496997,6,706,265.56,3897.0,-0.046269,0.097464,-0.068513
46338,2022-09-24,47,沖縄県,338,497335,5,711,243.48,3573.0,-0.471049,0.035169,-0.489426
46339,2022-09-25,47,沖縄県,533,497868,0,711,243.41,3572.0,0.576923,0.145208,-0.001873
46340,2022-09-26,47,沖縄県,284,498152,3,714,242.73,3562.0,-0.467167,0.142676,-0.034014
46341,2022-09-27,47,沖縄県,568,498720,0,714,259.90,3814.0,1.000000,0.274843,0.797468


In [7]:
latest = pref.loc[pref["日付"]==pref["日付"].max()][["日付","都道府県名","各地の感染者数_1日ごとの発表数","各地の死者数_1日ごとの発表数","直近1週間の感染者数","新規感染者数前日比","直近1週間の新規感染者数前日比平均","新規感染者数先週比"]]
latest.head()

,日付,都道府県名,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
985,2022-09-27,北海道,2560,5,17692.0,0.074276,0.135760,0.876833
1971,2022-09-27,青森県,548,0,2871.0,0.602339,0.218592,1.468468
2957,2022-09-27,岩手県,420,0,3009.0,0.063291,0.131010,0.666667
3943,2022-09-27,宮城県,820,1,5871.0,0.595331,0.558210,1.336182
4929,2022-09-27,秋田県,730,1,4041.0,1.500000,0.734954,2.364055


In [8]:
all_jp = pd.merge(all_jp, latest, on="都道府県名")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
0,三重県,23664,251235,524,98505.01,2022-09-27,393,1,5657.0,-0.548794,0.475531,0.122857
1,京都府,25636,475063,1051,128234.34,2022-09-27,840,3,7154.0,-0.129534,0.222997,0.953488
2,佐賀県,40426,156414,221,134226.45,2022-09-27,323,1,2445.0,1.182432,0.374990,0.362869
3,兵庫県,27608,1003778,2845,127939.31,2022-09-27,999,1,14620.0,-0.363289,0.175354,0.074194
4,北海道,986,740518,2580,98189.06,2022-09-27,2560,5,17692.0,0.074276,0.135760,0.876833


## 人口データを付与

In [9]:
pop = pd.read_excel("COVID-19-jp/2001stjin.xls",header=3)[["Unnamed: 1","計"]]
pop.rename(columns={"Unnamed: 1":"都道府県名"},inplace=True)
pop.rename(columns={"計":"人口"},inplace=True)
pop = pop.loc[pop["都道府県名"]!="合計"].reset_index(drop=True)

In [10]:
all_jp = pd.merge(all_jp, pop, on="都道府県名", how="left")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口
0,三重県,23664,251235,524,98505.01,2022-09-27,393,1,5657.0,-0.548794,0.475531,0.122857,1813859
1,京都府,25636,475063,1051,128234.34,2022-09-27,840,3,7154.0,-0.129534,0.222997,0.953488,2545899
2,佐賀県,40426,156414,221,134226.45,2022-09-27,323,1,2445.0,1.182432,0.374990,0.362869,823810
3,兵庫県,27608,1003778,2845,127939.31,2022-09-27,999,1,14620.0,-0.363289,0.175354,0.074194,5549568
4,北海道,986,740518,2580,98189.06,2022-09-27,2560,5,17692.0,0.074276,0.135760,0.876833,5267762


## 人口当たりの感染者数

In [11]:
all_jp["１万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 10000)
all_jp["10万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 100000)
all_jp["直近1週間の10万人当たりの感染者数"] = all_jp["直近1週間の感染者数"] / (all_jp["人口"] / 100000)
all_jp.sort_values("都道府県コード")

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
4,北海道,986,740518,2580,98189.06,2022-09-27,2560,5,17692.0,0.074276,0.135760,0.876833,5267762,1405.754474,14057.544741,335.854201
41,青森県,1972,162460,293,91154.00,2022-09-27,548,0,2871.0,0.602339,0.218592,1.468468,1275783,1273.414052,12734.140524,225.038271
19,岩手県,2958,107414,191,61388.75,2022-09-27,420,0,3009.0,0.063291,0.131010,0.666667,1235517,869.385043,8693.850429,243.541772
11,宮城県,3944,257027,391,77512.52,2022-09-27,820,1,5871.0,0.595331,0.558210,1.336182,2292385,1121.220912,11212.209118,256.108812
36,秋田県,4930,99413,222,71430.99,2022-09-27,730,1,4041.0,1.500000,0.734954,2.364055,985416,1008.842966,10088.429658,410.080616
15,山形県,5916,104587,194,67488.32,2022-09-27,749,1,4026.0,1.074792,0.296537,1.215976,1082296,966.343773,9663.437729,371.986961
35,福島県,6902,185287,266,69924.80,2022-09-27,783,0,5349.0,0.199081,0.095814,0.410811,1881981,984.531725,9845.317248,284.221785
38,茨城県,7888,360021,676,87205.52,2022-09-27,556,4,7413.0,-0.558029,0.433755,-0.007143,2921436,1232.342588,12323.425877,253.745076
27,栃木県,8874,222467,488,79851.36,2022-09-27,750,0,5279.0,0.630435,0.322277,0.773050,1965516,1131.850364,11318.503640,268.580871
37,群馬県,9860,234284,465,83849.41,2022-09-27,564,2,5657.0,-0.088853,0.123608,0.005348,1969439,1189.597647,11895.976468,287.239158


## 感染者数

### 平均値、中央値

In [12]:
all_jp["各地の感染者数合計"].describe()

count    4.700000e+01
mean     4.498510e+05
std      5.907189e+05
min      6.238300e+04
25%      1.489550e+05
50%      2.224670e+05
75%      4.513140e+05
max      3.154675e+06
Name: 各地の感染者数合計, dtype: float64

### TOP 10

In [13]:
all_jp.sort_values("各地の感染者数合計", ascending=False).reset_index(drop=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
0,東京都,12818,3154675,5798,156489.67,2022-09-27,5247,6,45507.0,-0.169253,0.103266,0.382973,13834925,2280.225589,22802.255885,328.928418
1,大阪府,26622,2086723,6412,164520.29,2022-09-27,3300,9,30731.0,0.462766,0.365431,0.551481,8849635,2357.976346,23579.763459,347.257260
2,神奈川県,13804,1478642,2964,111394.31,2022-09-27,2571,3,23295.0,-0.118917,0.006835,-0.075845,9209442,1605.571760,16055.717599,252.946921
3,愛知県,22678,1357704,2862,125276.80,2022-09-27,2713,7,24014.0,0.748067,0.484186,0.588407,7575530,1792.223118,17922.231184,316.994323
4,埼玉県,10846,1183785,2363,111945.72,2022-09-27,2525,5,22625.0,-0.340903,0.200970,0.416153,7390054,1601.862449,16018.624492,306.154732
5,福岡県,39440,1074697,1935,146030.79,2022-09-27,798,6,13211.0,-0.167015,0.076499,-0.469767,5129841,2094.990858,20949.908584,257.532348
6,兵庫県,27608,1003778,2845,127939.31,2022-09-27,999,1,14620.0,-0.363289,0.175354,0.074194,5549568,1808.749798,18087.497982,263.443929
7,千葉県,11832,942199,2550,104182.73,2022-09-27,1663,10,18783.0,-0.409026,0.054600,-0.022914,6319772,1490.874987,14908.749873,297.210089
8,北海道,986,740518,2580,98189.06,2022-09-27,2560,5,17692.0,0.074276,0.135760,0.876833,5267762,1405.754474,14057.544741,335.854201
9,沖縄県,46342,498720,714,237457.34,2022-09-27,568,0,3814.0,1.000000,0.274843,0.797468,1481547,3366.211129,33662.111293,257.433615


## １万人あたりの感染者数

### 平均値、中央値

In [14]:
all_jp["１万人当たりの感染者数"].describe()

count      47.000000
mean     1481.261470
std       441.976902
min       869.385043
25%      1187.141814
50%      1408.137155
75%      1628.163389
max      3366.211129
Name: １万人当たりの感染者数, dtype: float64

## 直近１週間の１０万人あたりの感染者数

In [15]:
all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
36,秋田県,4930,99413,222,71430.99,2022-09-27,730,1,4041.0,1.500000,0.734954,2.364055,985416,1008.842966,10088.429658,410.080616
12,宮崎県,44370,194341,358,126286.28,2022-09-27,391,2,4132.0,-0.208502,0.263418,0.401434,1095903,1773.341254,17733.412537,377.040669
29,滋賀県,24650,232024,317,113973.41,2022-09-27,620,2,5289.0,0.409091,0.090110,0.093474,1420948,1632.881710,16328.817100,372.216295
15,山形県,5916,104587,194,67488.32,2022-09-27,749,1,4026.0,1.074792,0.296537,1.215976,1082296,966.343773,9663.437729,371.986961
21,広島県,33524,427565,752,105819.68,2022-09-27,1125,5,10485.0,-0.397752,0.206981,0.254181,2826858,1512.509649,15125.096485,370.906498
30,熊本県,42398,329032,633,131672.44,2022-09-27,637,4,6403.0,0.546117,0.488572,0.814815,1769880,1859.063891,18590.638913,361.775940
18,岡山県,32538,272840,404,100173.10,2022-09-27,561,1,6741.0,-0.480074,0.059117,-0.110935,1903627,1433.263975,14332.639745,354.113490
9,大阪府,26622,2086723,6412,164520.29,2022-09-27,3300,9,30731.0,0.462766,0.365431,0.551481,8849635,2357.976346,23579.763459,347.257260
44,高知県,38454,99691,294,100035.10,2022-09-27,319,1,2422.0,0.812500,0.469761,1.155405,709230,1405.623000,14056.229996,341.497117
4,北海道,986,740518,2580,98189.06,2022-09-27,2560,5,17692.0,0.074276,0.135760,0.876833,5267762,1405.754474,14057.544741,335.854201


## 直近１週間の新規感染者 増加速度

## 増加速度 統計情報

In [16]:
all_jp["直近1週間の新規感染者数前日比平均"].describe()

count    47.000000
mean      0.283310
std       0.175584
min       0.006835
25%       0.133385
50%       0.263418
75%       0.424839
max       0.734954
Name: 直近1週間の新規感染者数前日比平均, dtype: float64

### WORST 10(増加速度の早い順)

In [17]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
36,秋田県,4930,99413,222,71430.99,2022-09-27,730,1,4041.0,1.500000,0.734954,2.364055,985416,1008.842966,10088.429658,410.080616
20,島根県,31552,81827,92,84563.94,2022-09-27,307,0,2105.0,0.860606,0.659691,1.192857,679324,1204.535685,12045.356855,309.866868
16,山梨県,18734,95387,153,81886.10,2022-09-27,213,0,1786.0,0.789916,0.591004,0.990654,826579,1153.997380,11539.973796,216.071301
11,宮城県,3944,257027,391,77512.52,2022-09-27,820,1,5871.0,0.595331,0.558210,1.336182,2292385,1121.220912,11212.209118,256.108812
42,静岡県,21692,490165,739,93707.47,2022-09-27,1399,2,10852.0,0.867824,0.511598,1.493761,3708556,1321.713896,13217.138962,292.620632
30,熊本県,42398,329032,633,131672.44,2022-09-27,637,4,6403.0,0.546117,0.488572,0.814815,1769880,1859.063891,18590.638913,361.775940
24,愛知県,22678,1357704,2862,125276.80,2022-09-27,2713,7,24014.0,0.748067,0.484186,0.588407,7575530,1792.223118,17922.231184,316.994323
0,三重県,23664,251235,524,98505.01,2022-09-27,393,1,5657.0,-0.548794,0.475531,0.122857,1813859,1385.085610,13850.856103,311.876502
44,高知県,38454,99691,294,100035.10,2022-09-27,319,1,2422.0,0.812500,0.469761,1.155405,709230,1405.623000,14056.229996,341.497117
10,奈良県,28594,219788,522,115197.02,2022-09-27,426,0,4392.0,-0.506373,0.436223,0.577778,1353837,1623.445068,16234.450676,324.411284


### BEST 10(増加速度の遅い順／減少速度の速い順)

In [18]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
32,神奈川県,13804,1478642,2964,111394.31,2022-09-27,2571,3,23295.0,-0.118917,0.006835,-0.075845,9209442,1605.571760,16055.717599,252.946921
5,千葉県,11832,942199,2550,104182.73,2022-09-27,1663,10,18783.0,-0.409026,0.054600,-0.022914,6319772,1490.874987,14908.749873,297.210089
18,岡山県,32538,272840,404,100173.10,2022-09-27,561,1,6741.0,-0.480074,0.059117,-0.110935,1903627,1433.263975,14332.639745,354.113490
34,福岡県,39440,1074697,1935,146030.79,2022-09-27,798,6,13211.0,-0.167015,0.076499,-0.469767,5129841,2094.990858,20949.908584,257.532348
29,滋賀県,24650,232024,317,113973.41,2022-09-27,620,2,5289.0,0.409091,0.090110,0.093474,1420948,1632.881710,16328.817100,372.216295
35,福島県,6902,185287,266,69924.80,2022-09-27,783,0,5349.0,0.199081,0.095814,0.410811,1881981,984.531725,9845.317248,284.221785
33,福井県,17748,113234,98,102376.67,2022-09-27,297,0,2600.0,-0.292857,0.097328,-0.003356,780053,1451.619313,14516.193130,333.310685
26,東京都,12818,3154675,5798,156489.67,2022-09-27,5247,6,45507.0,-0.169253,0.103266,0.382973,13834925,2280.225589,22802.255885,328.928418
23,愛媛県,37468,161324,288,83876.82,2022-09-27,477,0,3809.0,0.180693,0.105943,0.104167,1369131,1178.294845,11782.948454,278.205665
37,群馬県,9860,234284,465,83849.41,2022-09-27,564,2,5657.0,-0.088853,0.123608,0.005348,1969439,1189.597647,11895.976468,287.239158


## グラフ化

In [19]:
import matplotlib.pyplot as plt
import japanize_matplotlib
%matplotlib notebook
#%matplotlib widget
#%matplotlib inline

In [20]:
all_jp_desc = all_jp.sort_values(["各地の感染者数合計"],ascending=False)
all_jp_desc_10k = all_jp.sort_values(["１万人当たりの感染者数"],ascending=False)
all_jp_desc_100k = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False)

In [21]:
fig = plt.figure(figsize=(9.5,5))
plt.subplot(1,2,1) # (rows, columns, panel number)
plt.title("各地の新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["各地の感染者数合計"])
plt.barh(chart_data["都道府県名"],chart_data["各地の感染者数合計"],align="center")

plt.subplot(1,2,2) # (rows, columns, panel number)
plt.title("1万人当たりの新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["１万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["１万人当たりの感染者数"])

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [22]:
fig = plt.figure(figsize=(9.5,10))

#plt.style.use("ggplot")
### 最新の各地の新規感染者数
plt.subplot(2,2,1) # (rows, columns, panel number)
plt.title("{}の各地の新規感染者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の感染者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の感染者数_1日ごとの発表数"])
for i,v in enumerate(chart_data["各地の感染者数_1日ごとの発表数"]):
    plt.text(v,i,"{:,}".format(v),va="center",fontsize=7)
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 最新の各地の死者数
plt.subplot(2,2,2) # (rows, columns, panel number)
plt.title("{}の各地の新規死者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の死者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の死者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")



### 直近1週間の10万人当たりの新規感染者数
plt.subplot(2,2,3) # (rows, columns, panel number)
plt.title("直近1週間の10万人当たりの新規感染者数")
chart_data = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["直近1週間の10万人当たりの感染者数"])
for i,v in enumerate(chart_data["直近1週間の10万人当たりの感染者数"]):
    plt.text(v,i,"{:,}".format(int(v)),va="center",fontsize=7)
plt.plot([15,15],[0,47],"--",lw=0.5,color="black")
plt.text(15,0,"Stage3",ha="center",va="top",fontsize=8)
plt.plot([25,25],[0,47],"--",lw=0.5,color="black")
plt.text(25,0,"Stage4",ha="center",va="top",fontsize=8)
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 新規感染者数の増加速度（新規感染者数前週同曜日比）
plt.subplot(2,2,4) # (rows, columns, panel number)
plt.title("新規感染者数前週同曜日比")
chart_data = all_jp.sort_values("新規感染者数先週比",na_position="first")
plt.barh(chart_data["都道府県名"], chart_data["新規感染者数先週比"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [23]:
fig.savefig("out/covid-domestic.png")

## 度数分布表

In [24]:
import numpy as np
data = np.array(all_jp["各地の感染者数合計"])
 
# ヒストグラム
hist, bin_edges = np.histogram(data, bins=8)

print(hist)
print(bin_edges)

[35  4  4  2  0  1  0  1]
[  62383.   448919.5  835456.  1221992.5 1608529.  1995065.5 2381602.
 2768138.5 3154675. ]


## ヒストグラム

In [25]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["各地の感染者数合計"],bins=[0,20000,40000,80000,100000,120000,140000,160000])

<IPython.core.display.Javascript object>

(array([0., 0., 1., 5., 3., 2., 2.]),
 array([     0,  20000,  40000,  80000, 100000, 120000, 140000, 160000]),
 <BarContainer object of 7 artists>)

In [26]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["10万人当たりの感染者数"],bins=[0,200,400,600,800,1000,1200])


<IPython.core.display.Javascript object>

(array([0., 0., 0., 0., 0., 0.]),
 array([   0,  200,  400,  600,  800, 1000, 1200]),
 <BarContainer object of 6 artists>)

In [27]:
pref[pref["都道府県名"]=="山口県"].tail(10)


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,各地の直近1週間の人口10万人あたりの感染者数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
34500,2022-09-18,35,山口県,593,157799,1,374,395.96,5314.0,-0.126657,0.023755,-0.329944
34501,2022-09-19,35,山口県,378,158177,2,376,388.66,5216.0,-0.362563,0.037982,-0.205882
34502,2022-09-20,35,山口県,231,158408,0,376,329.94,4428.0,-0.388889,-0.180539,-0.773307
34503,2022-09-21,35,山口県,873,159281,2,378,325.25,4365.0,2.779221,0.228128,-0.067308
34504,2022-09-22,35,山口県,725,160006,0,378,316.45,4247.0,-0.169530,0.218104,-0.139976
34505,2022-09-23,35,山口県,636,160642,1,379,306.62,4115.0,-0.122759,0.213277,-0.171875
34506,2022-09-24,35,山口県,379,161021,2,381,284.26,3815.0,-0.404088,0.172105,-0.441826
34507,2022-09-25,35,山口県,497,161518,1,382,277.11,3719.0,0.311346,0.234677,-0.161889
34508,2022-09-26,35,山口県,361,161879,0,382,275.84,3702.0,-0.273642,0.247380,-0.044974
34509,2022-09-27,35,山口県,409,162288,3,385,289.11,3880.0,0.132964,0.321930,0.770563
